# **Εργασία 1**

Παλινδρόμηση με νευρωνικά δίκτυα και το climate change dataset.

## Φόρτωση και προεργασία

Φόρτωση βιβλιοθηκών, φόρτωση και απεικόνιση δεδομένων

In [ ]:
# Εισάγουμε όλες τις απαραίτητες βιβλιοθήκες
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Φορτώνουμε το αρχείο CSV με τη βιβλιοθήκη Pandas μόνο για λόγους απεικόνισης
dataframe = pd.read_csv('climate_change.csv')
print(dataframe.head(5))
print("Διαστάσεις:", dataframe.shape)

   Year  Month    MEI     CO2      CH4      N2O   CFC-11   CFC-12        TSI  \
0  1983      5  2.556  345.96  1638.59  303.677  191.324  350.113  1366.1024   
1  1983      6  2.167  345.52  1633.71  303.746  192.057  351.848  1366.1208   
2  1983      7  1.741  344.15  1633.22  303.795  192.818  353.725  1366.2850   
3  1983      8  1.130  342.25  1631.35  303.839  193.602  355.633  1366.4202   
4  1983      9  0.428  340.17  1648.40  303.901  194.392  357.465  1366.2335   

   Aerosols   Temp  
0    0.0863  0.109  
1    0.0794  0.118  
2    0.0731  0.137  
3    0.0673  0.176  
4    0.0619  0.149  
Διαστάσεις: (308, 11)


## Δημιουργία custom αντικειμένου Dataset

Δημιουργούμε ένα αντικείμενο Dataset το οποίο θα τροφοδοτήσει τους dataloaders, και ένα αντικείμενο transform που θα τροποποιεί τα δεδομένα πριν διαβαστούν από τους loaders.

Στο μέλλον το transform θα είναι πολύ σημαντικό για το data augmentation, αλλά για την ώρα το χρησιμοποιούμε για την κανονικοποίηση των δεδομένων.

In [ ]:
class ClimateDataset(torch.utils.data.Dataset):
  def __init__(self, csv_file, transform=None, train=True):
    """
    Args:
      csv_file (string): Path to the csv file
      transform (callable, optional): Optional transform to be applied
          on each sample.
      train (bool): whether to create the training set or the test set
    """

    # Ορίζουμε το μέγεθος του training set
    train_set_size = 250

    # Φόρτωση του csv μέσω Pandas, διαγραφή στηλών που δε βοηθούν στην ανάλυση,
    # μετατροπή των Pandas Dataframes σε Numpy Arrays, που έχουν πιο βοηθητικές
    # ιδιότητες
    dataframe = pd.read_csv(csv_file)
    targets = np.array(dataframe[['Temp']]).astype('float32')
    data = np.array(dataframe.drop(columns=['Year','Month','Temp'])).astype('float32')

    # Διαχωρισμός των δεδομένων σε training και test set
    # Αυτή η γραμμή θα πρέπει στο μέλλον να αντικατασταθεί αν θέλουμε να κάνουμε 
    # πιο ρεαλιστική αξιολόγηση
    # Θέτουμε και το seed στο random state για λόγους επαναληψιμότητας
    train_data, test_data, train_targets, test_targets = train_test_split(data, 
                            targets, train_size=train_set_size, random_state=665)

    # Θέτουμε και αρχικοποιούμε το transform
    self.transform = transform
    # Προσοχή, το transform αρχικοποιείται με τις τιμές του training set, είτε
    # το Dataset αφορά το train είτε το test set.
    self.transform.fit(train_data, train_targets)

    # Αν μας έχει ζητηθεί να δημιουργηθεί train database, κρατάμε τα train data.
    # Αλλιώς κρατάμε τα test data. 
    if train==True:
      self.data = train_data
      self.targets = train_targets
    else:
      self.data = test_data
      self.targets = test_targets
    # Η αρχικοποίηση τελειώνει εδώ




  # Ή __len__ ενός αντικειμένου Dataset οφείλει να επιστρέφει το πλήθος των
  # αντικειμένων του.
  def __len__(self):
    return len(self.targets)

  # Η __getitem__ ενός αντικειμένου Dataset παίρνει ως παράμετρο ένα index και 
  # επιστρέφει το αντίστοιχο αντικείμενο. Τη χρησιμοποιεί ο DataLoader για να 
  # αντλεί minibatches.
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    item_data = self.data[idx,:]
    item_target = self.targets[idx,:]
    
    # Αν έχουμε ορίσει transform, το εφαρμόζουμε στο αντικείμενο πριν το 
    # επιστρέψουμε. Όταν καλώ το transform με το όνομά του, στην πραγματικότητα
    # καλείται η transform.__call__() (βλ. παρακάτω)
    if self.transform:
      item_data, item_target = self.transform(item_data, item_target)
    return item_data, item_target





# Η κλάση αυτή θα παίξει το ρόλο του Transform. Θα αφαιρεί την ελάχιστη τιμή
# από κάθε στήλη των δεδομένων και θα διαιρεί με το φάσμα της, ώστε να κανονικοποιεί
# τις τιμές όλων των μεταβλητών (και των στόχων) στο [0,1]
class MinMaxScaler():

  # H fit αρχικοποιείται με τα training data, και υπολογίζει τις ελάχιστες και 
  # μέγιστες τιμές. Τα αντικείμενα είναι numpy arrays, οπότε έχουν ενσωματωμένη
  # τη συνάρτηση min και max 
  def fit(self, data, targets):
    self.data_min = data.min(0, keepdims=True)
    self.target_min = targets.min(0, keepdims=True)
    self.data_max = data.max(0, keepdims=True)
    self.target_max = targets.max(0, keepdims=True)
    return self

  # Όταν καλείται η transform.__call__(), δέχεται ένα αντικείμενο (data και target)
  # και το επιστρέφει μετασχηματισμένο -εν προκειμένω κανονικοποιημένο
  def __call__(self, data, target):
    data = (data - self.data_min)/(self.data_max-self.data_min)
    target = (target - self.target_min)/(self.target_max-self.target_min)
    return data, target

  # Θα εκπαιδεύσουμε ένα σύστημα να μαθαίνει τα κανονικοποιημένα targets. Για
  # εφαρμογή στον πραγματικό κόσμο, θα χρειαστεί να μπορούμε να μετασχηματίζουμε
  # τα outputs του συστήματος πίσω στο αρχικό φάσμα τιμών του
  def inverse_transform(self, data, target):
    data=data * (self.data_max-self.data_min) + self.data_min
    target = target * (self.target_max-self.target_min) + self.target_min
    return data, target

## Δημιουργία DataLoaders

Δημιουργούμε dataloaders για το training set και για το test set.

In [ ]:
# To batch size είναι αρκετά μεγάλο ώστε να χωράει όλα τα training data και όλα 
# τα test data σε ένα loop (πρακτικά εφαρμόζουμε Batch Gradient Descent, κάθε
# Batch είναι ένα Epoch) 
# (αν το batch size είναι μεγαλύτερο από τα διαθέσιμα δεδομένα, η pytorch 
# δημιουργεί ένα batch με τα διαθέσιμα δεδομένα)
batch_size = 500
transform=MinMaxScaler()

train_dataset = ClimateDataset(csv_file='climate_change.csv', train=True, transform=transform)
test_dataset = ClimateDataset(csv_file='climate_change.csv', train=False, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

## Αρχιτεκτονική δικτύου

Ορίζουμε ένα απλό δίκτυο με έναν νευρώνα για γραμμική παλινδρόμηση

In [ ]:
# Κώδικας ορισμού του δικτύου
class NeuralNetwork1(nn.Module):
  def __init__(self):
    # Τρέχω την init της κλάσης-γονιού
    super(NeuralNetwork1, self).__init__()
    
    #Δημιουργώ μια αρχιτεκτονική που θα αποτελείται αποκλειστικά από έναν νευρώνα με γραμμική συνάρτηση ενεργοποίησης.
    self.network_architecture = nn.Sequential(
        nn.Linear(8, 1),
        #Το input 𝐱 του δικτύου αποτελειται απο 8 μεταβλητες, καθως κατα την προεργασία από την κλάση ClimateDataset αφαιρεσαμε τα targets, 'Temp', και τις στηλες 'Year' και 'Month'.
        )
    
  def forward(self, x):
    logits = self.network_architecture(x)
    return logits

## Training και Test Loop

Γράφουμε τον κώδικα του training loop και του test loop

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, display):
    size = len(dataloader.dataset)
    
    
    for batch, (X, y) in enumerate(dataloader):
      # Κάθε batch έχει διαστάσεις [250, 1, 8]
      # Compute prediction and loss
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if display % 100 == 0:
        loss, current = loss.item(), len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    TheTargets = []
    ThePredictions = []

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
    for i in y:
      TheTargets.append(i.item())
    
    for j in pred:
      ThePredictions.append(j.item())


    test_loss /= num_batches
    correct /= size
  
    print(f"Test Error: \n R2 Score: {r2_score(TheTargets, ThePredictions)} , Avg loss: {test_loss:>8f} \n")

### Συνάρτηση κόστους (Loss Function)

In [ ]:
# Ορίζουμε ως Loss Function την MSELoss() (Μέσο Τετραγωνικό Σφάλμα) καθώς ειναι κατάλληλη για προβληματα παλινδρόμησης.
loss_fn = nn.MSELoss()

## Ορισμός υπερπαραμέτρων και εκπαίδευση δικτύου

Κώδικας εκτέλεσης και αξιολόγησης του δικτύου 

In [ ]:
# Το manual_seed καθορίζει την τυχαία αρχικοποίηση των βαρών/παραμέτρων του δικτύου
# Πρόκειται απλώς για ένα σταθερό seed στο random number generator της pytorch.
torch.manual_seed(100)
#torch.manual_seed(50)

model = NeuralNetwork1()
learning_rate = 0.001
epochs = 3000

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Κώδικας εκπαίδευσης εδώ:
for t in range(epochs):
    if t % 100 == 0:
      print(f"Epoch {t}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, t)
    if t % 100 == 0:
      test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 0
-------------------------------
loss: 0.532136  [  250/  250]
Test Error: 
 R2 Score: -25.27130098504746 , Avg loss: 0.551750 

Epoch 100
-------------------------------
loss: 0.162329  [  250/  250]
Test Error: 
 R2 Score: -6.7977772675125765 , Avg loss: 0.163769 

Epoch 200
-------------------------------
loss: 0.068843  [  250/  250]
Test Error: 
 R2 Score: -2.2233631332819366 , Avg loss: 0.067697 

Epoch 300
-------------------------------
loss: 0.044570  [  250/  250]
Test Error: 
 R2 Score: -1.0770251520030025 , Avg loss: 0.043622 

Epoch 400
-------------------------------
loss: 0.037668  [  250/  250]
Test Error: 
 R2 Score: -0.7669670089043794 , Avg loss: 0.037110 

Epoch 500
-------------------------------
loss: 0.035157  [  250/  250]
Test Error: 
 R2 Score: -0.6576277124832313 , Avg loss: 0.034813 

Epoch 600
-------------------------------
loss: 0.033786  [  250/  250]
Test Error: 
 R2 Score: -0.5958033873339577 , Avg loss: 0.033515 

Epoch 700
--------------------

### Τι παρατηρείτε να συμβαίνει στα train/test loss και στο R2 score καθώς περνάνε τα Epochs;

Απάντηση: Καθώς περνάνε τα Epochs μπορουμε να παρατηρήσουμε ότι όσον αφορά το train Loss αυτό μειώνεται συνεχως, ξεκινώντας απο το 1ο epoch βλεπουμε οτι το Loss ειναι 0.532136 και στην συνεχεια παρουσιάζει μεγάλη μείωση μέχρι να φτασει στο 0.029354 και να καταληξει στο 0.020415.

Όσον αφορα το test Loss μπορούμε να παρατήρησουμε ότι και αυτο μειώνεται σταδιακά παρουσιάζοντας μεγάλη μειωση στα πρώτα 400 epochs ωσπου να καταλήξει στο 0.019058.

Το R2 score παρατηρούμε ότι και αυτο αυξάνεται. Έχοντας αρχικά αρνητικές τιμές (<0) πράγμα που σημαίνει ότι το μοντελο δεν είναι αρκετα καλό.
Μετά απο 2500 epochs παρατηρούμε ότι η τιμή του R2 Score ειναι πλέον μεγαλύτερη του 0. Τελος, έχοντας R2 Score: **0.09256793853624656**, συμπεραίνουμε ότι το ποσοστό της διασποράς των δεδομένων που εκφράζεται σωστά από το μοντέλο ειναι αρκετα μικρό καθως φτανει μολις στο 0.09  

### Τι παρατηρείτε σε σχέση με τις τιμές των μετρικών στο χρόνο; Τι μπορείτε να συμπεράνετε από αυτό;

Απάντηση:
> Για torch.manual_seed(100): Ισχύει ότι προανέφερα παραπάνω για τα train/test loss και R2 Score. Συμπαιρένουμε ότι οι τιμές των μετρικών μειώνονται στην δειάρκεια του χρόνου με ιδιαίτερη μεταβολή στην αρχη καθως τα weights απο random που ειναι αρχικα μετα αρχίζουν σταδιακα να βελτιώνονται.



> Για torch.manual_seed(50): Παρατηρούμε ότι οι τιμές των μετρικών train, test Loss και R2 Score συνεχώς μειώνονται.Επιπλέον, η βελτίωση των τιμών σε σχεση με πριν ειναι μεγαλύτερη. Ειδικότερα το R2 Score βλεπουμε οτι εχει μεγαλυτερη βελτιωση στην τιμή του σε σχεση με πριν και αυξηση της τιμης εως και **0.31448371739472414** που ειναι συγκριτικα καλυτερη με πριν. Παρολαυτα, μπορουμε να παρατηρησδουμε επισης ότι στα πρωτα epochs οι τιμες των μετρικων ειναι αρκετα χειροότερες σε σχέση με πριν.


## **Optimizer: Adam** 
### Τι παρατηρείτε για την εκπαίδευση του δικτύου;

Απάντηση: Έχοντας εφαρμόσει ως Adam optimizer αντι για SGD παρατηρούμε, με βάση τις τιμες των μετρικων, ότι η εκπαιδευση του δικτύου βελτιώθηκε αρκετά καθώς οι τιμες και των τρειων μετρικων από epoch σε epoch βελτιώνονται ραγδαια με αποτελεσμα να έχουμε στο τελος **Train Loss: 0.009197, Avg loss: 0.007105 και R2 Score: 0.6616853195249721** που είναι μια αρκετα καλή τιμή πάνω απο το 0.5 που πλησιάζει το 1.

# 3. Νευρωνικό δίκτυο

In [ ]:
# Κώδικας ορισμού του δικτύου
class NeuralNetwork2(nn.Module):
  def __init__(self):
    # Τρέχω την init της κλάσης-γονιού
    super(NeuralNetwork2, self).__init__()
    
    #Δημιουργώ μια αρχιτεκτονική που θα αποτελείται αποκλειστικά από έναν νευρώνα με γραμμική συνάρτηση ενεργοποίησης.
    self.network_architecture = nn.Sequential(
        nn.Linear(8, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, 1),
        #η έξοδος του δικτύου είναι μη γραμμική συνάρτηση των αρχικων εισόδων (του δικτύου), καθώς όλοι οι κρυφοί νευρώνες έχουν μη γραμμική συνάρτηση ενεργοποίησης (ReLU).

    )
    
  def forward(self, x):
    logits = self.network_architecture(x)
    return logits

### Ορισμός υπερπαραμέτρων και εκπαίδευση δικτύου

In [ ]:
torch.manual_seed(50)

model = NeuralNetwork2()
learning_rate = 0.001
epochs = 3000

optimizer2 = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer2 = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Κώδικας εκπαίδευσης εδώ:
for t in range(epochs):
    if t % 100 == 0:
      print(f"Epoch {t}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer2, t)
    if t % 100 == 0:
      test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 0
-------------------------------
loss: 0.382369  [  250/  250]
Test Error: 
 R2 Score: -19.83784899013452 , Avg loss: 0.437636 

Epoch 100
-------------------------------
loss: 0.088239  [  250/  250]
Test Error: 
 R2 Score: -3.925405634362586 , Avg loss: 0.103443 

Epoch 200
-------------------------------
loss: 0.029093  [  250/  250]
Test Error: 
 R2 Score: -0.4570168839710582 , Avg loss: 0.030600 

Epoch 300
-------------------------------
loss: 0.018007  [  250/  250]
Test Error: 
 R2 Score: 0.30938529505681456 , Avg loss: 0.014504 

Epoch 400
-------------------------------
loss: 0.016027  [  250/  250]
Test Error: 
 R2 Score: 0.49129072559192677 , Avg loss: 0.010684 

Epoch 500
-------------------------------
loss: 0.015555  [  250/  250]
Test Error: 
 R2 Score: 0.5452724837616237 , Avg loss: 0.009550 

Epoch 600
-------------------------------
loss: 0.015322  [  250/  250]
Test Error: 
 R2 Score: 0.5669194103016943 , Avg loss: 0.009096 

Epoch 700
-----------------------

### Τι παρατηρείτε σε σχέση με την ταχύτητα εκπαίδευσης, και τις τιμές των κριτηρίων αξιολόγησης στο χρόνο;

Με Optimizer:

> **SGD**: Παρατηρούμε ότι η ταχύτητα εκπάιδευσης του δικτύου ειναι αρκετα υψηλης σε σχεση με πριν (βλ 2. Μεμονωμένος νευρώνας) και αυτο ειναι εμφανες απο τις χαμηλες τιμες των κριτηρίων αξιολόγησης (Train/Test Loss) απο το πρωτο κιολας epoch και καθως προχωραμε απο epoch σε epoch. Μπορει επιπλεον να παρατηρησει κανεις οτι η υψηλη ταχυτητα εκπαιδευσης ειναι ιδιαιτερα εμφανης στα πρωτα 500 epoch. Επιπλεόν οι τιμες των κριτηριων αξιολογησης, **Train και Test Loss**, είναι καλυτερες σε σχεση με πριν, **0.012835 και 0.007218** αντιστοιχα. η τιμη του **R2 Score** ειναι αρκετα πιο υψηλη (**0.6563155584689719**) σε σχεση με την *προηγουμενη τιμή (0.09256793853624656).*


> **Adam**: Παρατηρουμε ότι οπως και με τον SGD η ταχυτητα ειναι αρκετα υψηλης σε σχεση με πριν καθως και όσο περναει ο χρονος οι τιμες των κριτηρίων αξιολόγησης αυξομειωνονται συνεχως. Επιπλέον, παρατηρουμε οτι με τον Adam Optimizer, οι τιμες των κριτηρίων αξιολόγησης γνωρίζουν ραγδαια βελτίωση στα πρωτα 100 epochs. Όμως, παρατηρηθηκε ότι ενω αρχικα οι τιμή του **R2 Score** ειναι αρκετα καλή (*0.7615950507301525*) στην συνεχεια οσο περνανε τα epochs η τιμη της χειροτερευει ωσπου να καταληξει στο **0.6858477112869089**.




